In [1]:
# Importing Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import re
import nltk
from nltk.corpus import stopwords # importing 'stopwords' to this notebook
from nltk.stem.porter import PorterStemmer ## stemming of words

import joblib

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data= pd.read_csv('../input/stock-price-and-news-realted-to-it/AppleNewsStock.csv')
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,News
0,2006-12-01,13.114285,13.190000,12.871428,91.320000,13.045714,198769900,WHAT'S ON TONIGHT : 8 P.M. (TLC) ASHLEY JUDD A...
1,2006-12-04,13.125714,13.150000,12.928572,91.120003,13.017143,177384200,More on Housing Prices : The broadest governme...
2,2006-12-05,13.092857,13.190000,12.981428,91.269997,13.038571,165709600,NaN
3,2006-12-06,12.948571,13.055715,12.810000,89.830002,12.832857,159546100,Honoring R.W. Apple in Words and Food : About ...
4,2006-12-07,12.861428,12.928572,12.414286,87.040001,12.434286,251206900,"Homebuilders, and Worries Over Jobs, Lead a De..."


# Data Prepration

In [3]:
# Assigning labels to the entries
label_list=[]
for i in range(1, len(data)):
    if data['Adj Close'][i] - data['Adj Close'][i-1]>= 0:
        label_list.append(1)
    else:
        label_list.append(0)
data1= data.iloc[1: ]
data1.insert(2, "Label", label_list)

data_new= data1[["Label", "News", "Open"]] # Making data with just news and labels


# Adding previous adj close
previous_adj_close= []
for i in range(1, len(data)):
    previous_adj_close.append(data["Adj Close"][i])
    
data_new.insert(3, "Pre Adj Close", previous_adj_close)

In [4]:
display(data_new)
# Setting New Index for data
data_new= data_new.set_index(i for i in range(0, len(data_new)))

,Label,News,Open,Pre Adj Close
1,0,More on Housing Prices : The broadest governme...,13.125714,13.017143
2,1,NaN,13.092857,13.038571
3,0,Honoring R.W. Apple in Words and Food : About ...,12.948571,12.832857
4,0,"Homebuilders, and Worries Over Jobs, Lead a De...",12.861428,12.434286
5,1,"Homebuilders, and Worries Over Jobs, Lead a De...",12.461429,12.608571
...,...,...,...,...
2512,0,NaN,111.360001,111.230003
2513,1,Fighting iOS Calendar Spam : Unsolicited invit...,111.129997,111.790001
2514,0,NaN,111.430000,111.570000
2515,0,NaN,110.779999,111.459999


In [5]:
# Finding Missing Values
miss_value_row_list=[]

for j in range(len(data_new)):
    if type(data_new["News"][j]) is str: # Non-missing values will have str type
        continue
    else: 
        print( 'Row ' + str(j))
        miss_value_row_list.append(j)
miss_value_row_list= list(set(miss_value_row_list)) # Removing repeating elements(row number) 
print('\n')
print("Row numbers with missing valus :" + str(miss_value_row_list))

Row 1
Row 6
Row 13
Row 42
Row 47
Row 57
Row 62
Row 77
Row 81
Row 83
Row 122
Row 128
Row 135
Row 141
Row 159
Row 168
Row 184
Row 213
Row 221
Row 254
Row 258
Row 265
Row 270
Row 278
Row 290
Row 297
Row 316
Row 320
Row 328
Row 351
Row 367
Row 372
Row 379
Row 393
Row 399
Row 417
Row 425
Row 434
Row 453
Row 463
Row 467
Row 472
Row 479
Row 482
Row 503
Row 511
Row 519
Row 521
Row 525
Row 531
Row 572
Row 584
Row 612
Row 618
Row 625
Row 633
Row 647
Row 652
Row 670
Row 682
Row 689
Row 696
Row 699
Row 703
Row 711
Row 731
Row 736
Row 749
Row 772
Row 775
Row 794
Row 834
Row 835
Row 857
Row 871
Row 899
Row 919
Row 942
Row 959
Row 972
Row 978
Row 987
Row 1027
Row 1035
Row 1043
Row 1047
Row 1053
Row 1065
Row 1078
Row 1088
Row 1095
Row 1107
Row 1111
Row 1121
Row 1124
Row 1130
Row 1145
Row 1151
Row 1161
Row 1168
Row 1172
Row 1179
Row 1185
Row 1195
Row 1201
Row 1208
Row 1214
Row 1222
Row 1226
Row 1231
Row 1237
Row 1243
Row 1274
Row 1296
Row 1298
Row 1301
Row 1328
Row 1380
Row 1404
Row 1430
Row 1448
Row 1

In [6]:
# Total Available Data:
len(data_new)-len(miss_value_row_list)

2322

In [7]:
# Removing rows with missing entries
data_new.drop(miss_value_row_list, inplace = True)

# Setting New Index for data
data_new= data_new.set_index(i for i in range(len(data_new)))

In [8]:
display(data_new.head())
display(data_new.tail())

,Label,News,Open,Pre Adj Close
0,0,More on Housing Prices : The broadest governme...,13.125714,13.017143
1,0,Honoring R.W. Apple in Words and Food : About ...,12.948571,12.832857
2,0,"Homebuilders, and Worries Over Jobs, Lead a De...",12.861428,12.434286
3,1,"Homebuilders, and Worries Over Jobs, Lead a De...",12.461429,12.608571
4,1,Sales of iPods and iTunes Not Much in Sync : T...,12.700000,12.678572


,Label,News,Open,Pre Adj Close
2317,0,Turn an iPad-Made Movie Into a DVD : Home vide...,109.809998,109.949997
2318,1,When Eve and Eve Bit the Apple : A Christian w...,109.720001,110.059998
2319,1,"Daily Report: At Apple, U.S. Jobs That Go Beyo...",110.120003,111.730003
2320,1,A Trade War Against China Might Be a Fight Tru...,111.949997,111.800003
2321,1,Fighting iOS Calendar Spam : Unsolicited invit...,111.129997,111.790001


In [9]:
len(data_new)

2322

# NLP

In [64]:
# Cleanig all the news(stacked together)
def nlp_preprocess(text):
# corpus=[]
    news= re.sub('[^a-zA-Z]', ' ', text)
    news= news.lower()
    news= news.split()
    news=[word for word in news if not word in set(stopwords.words('english'))]
    pe = PorterStemmer()
    news=[pe.stem(word) for word in news if not word in set(stopwords.words('english'))]
    news= ' '.join(news)
    # corpus.append(news)
    return news

In [65]:
# Target Variable
Y= data_new["Label"].values
display(Y)

# Input Variable
txt= data_new["News"].values

array([0, 0, 0, ..., 1, 1, 1])

In [66]:
#Spliting Dataset into Training and Test sets
from sklearn.model_selection import train_test_split
X_train_raw, X_test_raw, Y_train, Y_test= train_test_split(txt, Y, test_size=0.10, random_state=0)

In [67]:
# Train data
train_corpus= []
for i in range(len(X_train_raw)):
    train_corpus.append(nlp_preprocess(X_train_raw[i]))
    
# Test data
test_corpus= []
for i in range(len(X_test_raw)):
    test_corpus.append(nlp_preprocess(X_test_raw[i]))

In [68]:
cv_model= joblib.load('../input/sentiment-material-3/cv_model.pkl')
# Train data
trainX = cv_model.transform(train_corpus).toarray()

# Test data
testX= cv_model.transform(test_corpus).toarray()

In [69]:
# Sentiments + Open + Pre Adj Close
dataset = pd.DataFrame()

# open_price= []
# for i in range(len(data_new)):
#     open_price.append(data_new["Open"][i]/10)
    
# previous_adj_close= []
# for i in range(len(data_new)):
#     previous_adj_close.append(data["Adj Close"][i]/10)



# Adding sentence sentiments
txt_sentiment= joblib.load('../input/sentiment-material-3/Txt_sentiment.pkl')
neg= []
pos= []
neu= []

# Train data
for i in range(trainX.shape[0]):
    pred= txt_sentiment.predict_proba(trainX[i].reshape(1, -1))
    neg.append(pred[0][0])
    pos.append(pred[0][1])
    neu.append(pred[0][2])

# Test data
for i in range(testX.shape[0]):
    pred= txt_sentiment.predict_proba(testX[i].reshape(1, -1))
    neg.append(pred[0][0])
    pos.append(pred[0][1])
    neu.append(pred[0][2])

dataset.insert(0, "Neg", neg)
dataset.insert(1, "Pos", pos)
dataset.insert(2, "Neu", neu)
# dataset.insert(3, "Open", open_price)
# dataset.insert(4, "Prev Adj Close", previous_adj_close)

#Spliting Dataset into Training and Test sets
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test= train_test_split(dataset, Y, test_size=0.10, random_state=0)

#Feature Scaling
from sklearn.preprocessing import MinMaxScaler
sc_X= MinMaxScaler() 
X_train= sc_X.fit_transform(X_train)
X_test= sc_X.transform(X_test)

joblib.dump(sc_X, "Feature_scaler.pkl") # Saving the scaler for future use

['Feature_scaler.pkl']

In [70]:
dataset

,Neg,Pos,Neu
0,2.688677e-06,0.050659,9.493380e-01
1,6.460461e-06,0.000848,9.991457e-01
2,2.378624e-01,0.762137,5.701936e-07
3,1.996891e-08,0.000008,9.999923e-01
4,1.270218e-08,0.000002,9.999978e-01
...,...,...,...
2317,3.727986e-04,0.540203,4.594247e-01
2318,8.735641e-02,0.912225,4.188772e-04
2319,3.540883e-05,0.002528,9.974370e-01
2320,4.471008e-03,0.269835,7.256940e-01


In [71]:
# Saving the data frame
dataset.to_csv('Apple_Historic_News_Sentiment.csv',index=False)

In [72]:
#Spliting Dataset into Training and Test sets
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test= train_test_split(dataset, Y, test_size=0.10, random_state=0)

# Classification Algorithms

In [73]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
classifier= LogisticRegression(random_state= 0)
classifier.fit(X_train, Y_train)

# Predicting the test set result
y_pred= classifier.predict(X_test)
display(y_pred)

# Making the Confusion matrix
from sklearn.metrics import confusion_matrix
cm= confusion_matrix(Y_test, y_pred)
display(cm)

Accuracy= (cm[0][0] + cm[1][1])/(cm[0][0] + cm[1][1] + cm[0][1] + cm[1][0])
display(Accuracy)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1])

array([[  8, 113],
       [  8, 104]])

0.48068669527896996

In [74]:
# Decision Tree  
from sklearn.tree import DecisionTreeClassifier
classifier= DecisionTreeClassifier(criterion= 'entropy', random_state= 0)
classifier.fit(X_train, Y_train)

# Predicting the test set result
y_pred= classifier.predict(X_test)
display(y_pred)

# Making the Confusion matrix
from sklearn.metrics import confusion_matrix
cm= confusion_matrix(Y_test, y_pred)
display(cm)

Accuracy= (cm[0][0] + cm[1][1])/(cm[0][0] + cm[1][1] + cm[0][1] + cm[1][0])
display(Accuracy)

array([0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1])

array([[52, 69],
       [45, 67]])

0.5107296137339056

In [75]:
# Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
classifier = GradientBoostingClassifier()
classifier = classifier.fit(X_train, Y_train)

# Predicting the test set result
y_pred= classifier.predict(X_test)
display(y_pred)

# Making the Confusion matrix
from sklearn.metrics import confusion_matrix
cm= confusion_matrix(Y_test, y_pred)
display(cm)

Accuracy= (cm[0][0] + cm[1][1])/(cm[0][0] + cm[1][1] + cm[0][1] + cm[1][0])
display(Accuracy)

array([0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1,
       0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1])

array([[41, 80],
       [37, 75]])

0.4978540772532189

In [76]:
# Kernel SVM
from sklearn.svm import SVC 
classifier= SVC(kernel='rbf', random_state= 0)
classifier.fit(X_train, Y_train)

# Predicting the test set result
y_pred= classifier.predict(X_test)
display(y_pred)

# Making the Confusion matrix
from sklearn.metrics import confusion_matrix
cm= confusion_matrix(Y_test, y_pred)
display(cm)

Accuracy= (cm[0][0] + cm[1][1])/(cm[0][0] + cm[1][1] + cm[0][1] + cm[1][0])
display(Accuracy)

array([1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1])

array([[  9, 112],
       [  5, 107]])

0.4978540772532189

In [77]:
# Multinomial NB
from sklearn.naive_bayes import MultinomialNB
classifier=MultinomialNB()
classifier.fit(X_train, Y_train)

# Predicting the test set result
y_pred= classifier.predict(X_test)
display(y_pred)

# Making the Confusion matrix
from sklearn.metrics import confusion_matrix
cm= confusion_matrix(Y_test, y_pred)
display(cm)

Accuracy= (cm[0][0] + cm[1][1])/(cm[0][0] + cm[1][1] + cm[0][1] + cm[1][0])
display(Accuracy)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

array([[  0, 121],
       [  0, 112]])

0.48068669527896996

In [78]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
classifier= KNeighborsClassifier(n_neighbors= 7, metric='minkowski', p=2, leaf_size=70, weights= 'distance', algorithm= 'brute')
classifier.fit(X_train, Y_train)

# Predicting the test set result
y_pred= classifier.predict(X_test)
display(y_pred)

# Making the Confusion matrix
from sklearn.metrics import confusion_matrix
cm= confusion_matrix(Y_test, y_pred)
display(cm)

Accuracy= (cm[0][0] + cm[1][1])/(cm[0][0] + cm[1][1] + cm[0][1] + cm[1][0])
display(Accuracy)

array([0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1])

array([[51, 70],
       [49, 63]])

0.4892703862660944

In [79]:
# SGDC
from sklearn.linear_model import SGDClassifier
classifier = SGDClassifier(loss='modified_huber', random_state=0)
classifier = classifier.fit(X_train, Y_train)

# Predicting the test set result
y_pred= classifier.predict(X_test)
display(y_pred)

# Making the Confusion matrix
from sklearn.metrics import confusion_matrix
cm= confusion_matrix(Y_test, y_pred)
display(cm)

Accuracy= (cm[0][0] + cm[1][1])/(cm[0][0] + cm[1][1] + cm[0][1] + cm[1][0])
display(Accuracy)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

array([[121,   0],
       [112,   0]])

0.51931330472103

In [80]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier 
classifier= RandomForestClassifier(n_estimators= 500, criterion= 'entropy', random_state= 0) # Try with diffrent numbers of n_estimators(n_estimators= number of trees)
classifier.fit(X_train, Y_train)

# Predicting the test set result
y_pred= classifier.predict(X_test)
display(y_pred)

# Making the Confusion matrix
from sklearn.metrics import confusion_matrix
cm= confusion_matrix(Y_test, y_pred)
display(cm)

Accuracy= (cm[0][0] + cm[1][1])/(cm[0][0] + cm[1][1] + cm[0][1] + cm[1][0])
display(Accuracy)

array([0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1])

array([[54, 67],
       [43, 69]])

0.5278969957081545

In [81]:
# Saving the classifier for future use 
joblib.dump(classifier, "Historic_Apple_News_Sentiment.pkl", compress= 1)

['Historic_Apple_News_Sentiment.pkl']